<a href="https://colab.research.google.com/github/hikaruyaku/KIKAGAKU-/blob/master/%E3%83%8B%E3%83%A5%E3%83%BC%E3%83%A9%E3%83%AB%E3%83%8D%E3%83%83%E3%83%88%E3%83%AF%E3%83%BC%E3%82%AF%E3%81%AE%E5%AE%9F%E8%A3%85(%E5%9F%BA%E7%A4%8E).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## データセットの準備

In [0]:
from sklearn.datasets import load_iris

# Iris データセットの読み込み
x, t = load_iris(return_X_y = True)

In [3]:
# 形の確認
x.shape, t.shape

((150, 4), (150,))

In [4]:
# 型の確認
type(x), type(t)

(numpy.ndarray, numpy.ndarray)

In [5]:
# データ型の確認
x.dtype, t.dtype

(dtype('float64'), dtype('int64'))

In [6]:
!pip install torch torchvision

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# データ型の変換
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.int64)

In [9]:
#型の確認
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [10]:
#データ型の確認
x.dtype, t.dtype

(torch.float32, torch.int64)

In [11]:
#入力変数と目的変数をまとめて、ひとつのオブジェクト dataset に変換
dataset = torch.utils.data.TensorDataset(x,t)
dataset

In [12]:
#型の確認
type(dataset)

torch.utils.data.dataset.TensorDataset

In [13]:
# (入力値、目標値)のようにタプルで収納されている。
dataset[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor(0))

In [14]:
# 型の確認
type(dataset[0])

tuple

In [15]:
# 1 サンプル目の入力値
dataset[0][0]

tensor([5.1000, 3.5000, 1.4000, 0.2000])

In [16]:
# 1 サンプル目の目標値
dataset[0][1]

tensor(0)

In [17]:
# サンプル数は len で取得可能
len(dataset)

150

In [0]:
# 各データセットのサンプル数を決定
# train : val ; test = 60% : 20% : 20%
n_train = int(len(dataset) * 0.6)
n_val = int(len(dataset) * 0.2)
n_test = len(dataset) - n_train - n_val

In [19]:
# それぞれのサンプル数を確認
n_train, n_val, n_test

(90, 30, 30)

In [0]:
# ランダムに分割を行うため、シードを固定して再現性を確保
torch.manual_seed(0)

# データセットの分割
train, val, test = torch.utils.data.random_split(dataset, [n_train, n_val, n_test])

In [21]:
# サンプル数の確認
len(train), len(val), len(test)

(90, 30, 30)

## ミニバッチ学習

In [0]:
# バッチサイズ
batch_size = 10

In [0]:
# shuffle はデフォルトで False のため、学習データのみ True に指定
train_loader = torch.utils.data.DataLoader(train, batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size)
test_loader = torch.utils.data.DataLoader(test, batch_size)

## モデルの定義

In [0]:
class Net(nn.Module):


  # 使用するオブジェクトを定義
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(4,4) # 全結合層"fc1"input:4->output:4
    self.fc2 = nn.Linear(4,3) # 全結合層"fc2"input:4->output:3

  # 順伝播
  def forward(self, x):
    x = self.fc1(x)
    x = F.relu(x)
    return x

In [0]:
# 乱数のシードを固定して再現性を確保
torch.manual_seed(0)

# インスタンス化
net = Net()

In [26]:
# モデルの確認
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

## 目的関数を選択

In [27]:
# 目的関数の設定
criterion = F.cross_entropy
criterion

<function torch.nn.functional.cross_entropy>

## 最適化手法の選択

In [28]:
# net.parameters()を展開
for parameter in iter(net.parameters()):
  print(parameter)

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)
Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)
Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)
Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)


In [29]:
# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)

## モデルの学習手順

```
for batch in train_loader:
  batch を利用した処理
```


In [30]:
# バッチサイズ分のサンプルの抽出
batch = next(iter(train_loader))
batch

[tensor([[6.4000, 3.2000, 5.3000, 2.3000],
         [6.4000, 2.7000, 5.3000, 1.9000],
         [5.9000, 3.0000, 4.2000, 1.5000],
         [6.9000, 3.1000, 5.4000, 2.1000],
         [5.5000, 2.4000, 3.8000, 1.1000],
         [5.6000, 2.7000, 4.2000, 1.3000],
         [5.8000, 2.7000, 5.1000, 1.9000],
         [6.9000, 3.1000, 4.9000, 1.5000],
         [6.3000, 2.3000, 4.4000, 1.3000],
         [7.2000, 3.0000, 5.8000, 1.6000]]),
 tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2])]

In [0]:
# 入力値と目標値に分割
x, t = batch

In [32]:
# 入力値の確認
x

tensor([[6.4000, 3.2000, 5.3000, 2.3000],
        [6.4000, 2.7000, 5.3000, 1.9000],
        [5.9000, 3.0000, 4.2000, 1.5000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [5.8000, 2.7000, 5.1000, 1.9000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [7.2000, 3.0000, 5.8000, 1.6000]])

In [33]:
# 目標値の確認
t

tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2])

In [34]:
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1926,  0.1341, -0.0099,  0.3964],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [35]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3000, -0.3390, -0.2177], requires_grad=True)

In [36]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)

In [37]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)

In [38]:
# 予測値の算出(非推奨)
y = net.forward(x)
y

tensor([[0.0000, 0.3559, 0.0000, 0.0000],
        [0.0000, 0.1303, 0.0000, 0.0000],
        [0.0000, 0.1191, 0.0000, 0.0000],
        [0.0000, 0.1659, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0574, 0.0000, 0.0000],
        [0.0000, 0.2478, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [39]:
# call method を用いた forward の計算(推奨)
y = net(x)
y

tensor([[0.0000, 0.3559, 0.0000, 0.0000],
        [0.0000, 0.1303, 0.0000, 0.0000],
        [0.0000, 0.1191, 0.0000, 0.0000],
        [0.0000, 0.1659, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0574, 0.0000, 0.0000],
        [0.0000, 0.2478, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]], grad_fn=<ReluBackward0>)

In [40]:
# 目的関数の計算 criterion の call メソッドを利用
loss = criterion(y,t)
loss

tensor(1.3980, grad_fn=<NllLossBackward>)

In [0]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.weight.grad

# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

# 全結合層 fc2 の重みに関する勾配
net.fc2.weight.grad

# 全結合層 fc2 のバイアスに関する勾配
net.fc2.bias.grad

In [0]:
# 勾配の算出
loss.backward()

In [43]:
# 全結合層 fc1 の重みに関する勾配
net.fc1.weight.grad

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0919, -0.0718,  0.0061,  0.0375],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [44]:
# 全結合層 fc1 のバイアスに関する勾配
net.fc1.bias.grad

tensor([ 0.0000, -0.0287,  0.0000,  0.0000])

In [0]:
# 全結合層 fc2 の重みに関する勾配
net.fc2.weight.grad

In [0]:
# 全結合層 fc2 のバイアスに関する勾配
net.fc2.bias.grad

In [0]:
# 勾配の情報を用いたパラメータの更新
optimizer.step()

In [48]:
# 全結合層 fc1 の重み
net.fc1.weight

Parameter containing:
tensor([[-0.0037,  0.2682, -0.4115, -0.3680],
        [-0.1834,  0.1413, -0.0105,  0.3927],
        [-0.0444,  0.1323, -0.1511, -0.0983],
        [-0.4777, -0.3311, -0.2061,  0.0185]], requires_grad=True)

In [49]:
# 全結合層 fc1 のバイアス
net.fc1.bias

Parameter containing:
tensor([ 0.1977,  0.3029, -0.3390, -0.2177], requires_grad=True)

In [50]:
# 全結合層 fc2 の重み
net.fc2.weight

Parameter containing:
tensor([[ 0.1816,  0.4152, -0.1029,  0.3742],
        [-0.0806,  0.0529,  0.4527, -0.4638],
        [-0.3148, -0.1266, -0.1949,  0.4320]], requires_grad=True)

In [51]:
# 全結合層 fc2 のバイアス
net.fc2.bias

Parameter containing:
tensor([-0.3241, -0.2302, -0.3493], requires_grad=True)

### GPUの設定方法

In [52]:
# 演算に使用できる GPU の有無を確認
torch.cuda.is_available()

True

In [55]:
# GPU の設定状況に基づいたデバイスの選択
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [56]:
# 指定したデバイスへのモデルの転送
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [57]:
# 指定したデバイスへの入力変数の転送
x = x.to(device)
x

tensor([[6.4000, 3.2000, 5.3000, 2.3000],
        [6.4000, 2.7000, 5.3000, 1.9000],
        [5.9000, 3.0000, 4.2000, 1.5000],
        [6.9000, 3.1000, 5.4000, 2.1000],
        [5.5000, 2.4000, 3.8000, 1.1000],
        [5.6000, 2.7000, 4.2000, 1.3000],
        [5.8000, 2.7000, 5.1000, 1.9000],
        [6.9000, 3.1000, 4.9000, 1.5000],
        [6.3000, 2.3000, 4.4000, 1.3000],
        [7.2000, 3.0000, 5.8000, 1.6000]], device='cuda:0')

In [58]:
# 指定したデバイスへの目的変数の転送
t = t.to(device)
t

tensor([2, 2, 1, 2, 1, 1, 2, 1, 1, 2], device='cuda:0')

In [0]:
# 勾配情報の初期化
optimizer.zero_grad()

In [60]:
# 初期化後の勾配情報
net.fc1.weight.grad

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')

In [61]:
net.fc1.bias.grad

tensor([0., 0., 0., 0.], device='cuda:0')

## モデルを学習

In [62]:
# エポックの数
max_epoch = 1

# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)

# 学習ループ
for epoch in range(max_epoch):

  for batch in train_loader:

    # バッチサイズ分のサンプルを抽出
    x, t = batch

    # 学習時に使用するデバイスへデータの転送
    x = x.to(device)
    t = t.to(device)

    # パラメータの購買を初期化
    optimizer.zero_grad()

    # 予測値の算出
    y = net(x)

    # 目標値と予測値から目的関数の値を導出
    loss = criterion(y, t)

    # 目的関数の値を表示して確認
    # item(): tensot.Tensor => float
    print('loss', loss.item())

    # 各パラメータの勾配を算出
    loss.backward()

    # 勾配の情報を用いたパラメータの更新
    optimizer.step()

loss 1.3980052471160889
loss 1.3477176427841187
loss 1.486327886581421
loss 1.1727179288864136
loss 1.2937837839126587
loss 1.1389936208724976
loss 1.0209020376205444
loss 0.9248183965682983
loss 0.8647987246513367


In [63]:
# dim=1 で行ごとの最大値に対する要素番号を取得(dim=0 は列ごと)
y_label = torch.argmax(y, dim=1)
y_label

tensor([0, 0, 3, 0, 0, 3, 3, 0, 0, 0], device='cuda:0',
       grad_fn=<NotImplemented>)

In [64]:
# 目的変数
t

tensor([0, 1, 2, 0, 0, 2, 2, 1, 0, 0], device='cuda:0')

In [65]:
# 値が一致しているか確認
y_label == t

tensor([ True, False, False,  True,  True, False, False, False,  True,  True],
       device='cuda:0')

In [66]:
# 値がTrue となる個数の総和
torch.sum(y_label == t)

tensor(5, device='cuda:0')

In [67]:
# int => float
torch.sum(y_label == t) * 1.0

tensor(5., device='cuda:0')

In [68]:
# 正解率
acc = torch.sum(y_label == t) * 1.0 / len(t)
acc

tensor(0.5000, device='cuda:0')

### ACCのモデルへの組み込み

In [69]:
# モデルの初期化
torch.manual_seed(0)

# モデルのインスタンス化とデバイスへの転送
net = Net().to(device)

# 最適化手法の選択
optimizer = torch.optim.SGD(net.parameters(),lr=0.1)

for epoch in range(max_epoch):
  for batch in train_loader:

    x,t = batch
    x = x.to(device)
    t = t.to(device)
    optimizer.zero_grad()
    y = net(x)
    loss = criterion(y,t)

    # New : 正解率の算出
    y_label = torch.argmax(y, dim=1)
    acc = torch.sum(y_label == t) * 1.0 / len(t)
    print("accuracy", acc)

    loss.backward()
    optimizer.step()

accuracy tensor(0.2000, device='cuda:0')
accuracy tensor(0.6000, device='cuda:0')
accuracy tensor(0.4000, device='cuda:0')
accuracy tensor(0.3000, device='cuda:0')
accuracy tensor(0.1000, device='cuda:0')
accuracy tensor(0.3000, device='cuda:0')
accuracy tensor(0.4000, device='cuda:0')
accuracy tensor(0.5000, device='cuda:0')
accuracy tensor(0.5000, device='cuda:0')


In [0]:
# 正解率の計算
def calc_acc(data_loader):
  with torch.no_grad():
    accs = []
    for batch in data_loader:
      x, t = batch
      x = x.to(device)
      t = t.to(device)
      y = net(x)

      y_label = torch.argmax(y, dim=1)
      acc = torch.sum(y_label == t) * 1.0 / len(t)
      accs.append(acc)
  # 全体の平均を算出
  avg_acc = torch.tensor(accs).mean()
  print("Accuracy: {:.1f}%".format(avg_acc * 100))

  return avg_acc

In [71]:
# 検証データで確認
calc_acc(val_loader)

Accuracy: 43.3%


tensor(0.4333)

In [72]:
# テストデータで確認
calc_acc(test_loader)

Accuracy: 33.3%


tensor(0.3333)